In [1]:
import pandas as pd
import numpy as np
import re

In [37]:
data.head()

,Unnamed: 0,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
0,0,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Shell: Cotton 99%, Spandex 1%"
1,1,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Pocket lining: Polyester 65%, Cotton 35%"
2,2,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Shell: Cotton 99%, Spandex 1%"
3,3,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Light denim blue,"Pocket lining: Polyester 65%, Cotton 35%"
4,4,1024256001,men_jeans_slim,Slim Jeans,19.99,2022-04-24-11-21:30,1024256,1,Denim blue,"Shell: Cotton 99%, Spandex 1%"


In [38]:

data.sample()

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
226,938875013,men_jeans_slim,Slim Tapered Jeans,39.99,2022-04-24-11-21:30,938875,13,White,"Cotton 99%, Spandex 1%"


In [107]:
#loading the data
data = pd.read_csv('/home/guilherme/Documentos/repos/ds_ao_dev/projeto_webscraping/df_raw.csv')

#delete the columns not necessery
data = data.drop(['Unnamed: 0' ],axis=1)

#product id

data = data.dropna(subset=['product_id'])

#product category

#product name
data['products_name']= data['products_name'].apply(lambda x: x.replace(' ','_').lower())
#products_price

#já fiz o tratamento anteriomente

#product datetime

data['scrapy_datetime']= pd.to_datetime(data['scrapy_datetime'])
#style id

#color id

#color name
data['color_name']= data['color_name'].apply(lambda x: x.replace(' ','_').replace('/','_').lower() if pd.notnull(x) else x)

#Composition

data['Composition']= data['Composition'].where(~data['Composition'].str.contains('Pocket lining:',na=False) ) 

data['Composition']= data['Composition'].where(lambda x: ~x.str.contains('Shell:',na=False) )

data['Composition'] = data['Composition'].where(lambda x: ~x.str.contains('Lining:',na=False) )

#drop duplicates
#first were making drop the 
data= data.drop_duplicates(subset= ['product_id', 'products_category', 'products_name', 'products_price',
       'scrapy_datetime', 'stlye_id', 'color_id', 'color_name'],keep='last')


      

#reset index
data = data.reset_index(drop=True)


#break composition by comma (separando as composições dos materiais por coluna)
df1 = data['Composition'].str.split(',',expand=True)


#Cotton| Spandex | Polyester |  Elastomultiester

data

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
0,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,dark_denim_blue,NaN
1,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,black,NaN
2,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,light_denim_blue,NaN
3,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,denim_blue,NaN
4,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,1,dark_blue,NaN
...,...,...,...,...,...,...,...,...,...
576,814631018,men_jeans_slim,freefit®_slim_jeans,49.99,2022-04-24 11:00:00-21:30,814631,18,light_gray,"Cotton 90%, Elastomultiester 8%, Spandex 2%"
577,814631018,men_jeans_slim,freefit®_slim_jeans,49.99,2022-04-24 11:00:00-21:30,814631,18,black_no_fade_black,"Cotton 90%, Elastomultiester 8%, Spandex 2%"
578,814631018,men_jeans_slim,freefit®_slim_jeans,49.99,2022-04-24 11:00:00-21:30,814631,18,dark_blue,"Cotton 90%, Elastomultiester 8%, Spandex 2%"
579,814631018,men_jeans_slim,freefit®_slim_jeans,49.99,2022-04-24 11:00:00-21:30,814631,18,white,"Cotton 90%, Elastomultiester 8%, Spandex 2%"


In [90]:
data.columns

Index(['product_id', 'products_category', 'products_name', 'products_price',
       'scrapy_datetime', 'stlye_id', 'color_id', 'color_name', 'Composition'],
      dtype='object')

In [73]:
df1.apply(lambda x: x.unique())

0    [nan, Cotton 99%, Cotton 98%, Pocket: Cotton 1...
1    [None,  Spandex 1%,  Spandex 2%,  Polyester 6%...
2                [None,  Spandex 1%,  Spandex 2%, nan]
dtype: object

In [98]:
df1[df1[0].isna()]

,0,1,2
0,NaN,None,None
1,NaN,None,None
2,NaN,None,None
3,NaN,None,None
4,NaN,None,None
...,...,...,...
567,NaN,None,None
568,NaN,None,None
569,NaN,None,None
570,NaN,None,None


In [101]:
data.loc[571,:]

product_id                          1048642001
products_category            men_jeans_regular
products_name            regular_bootcut_jeans
products_price                           39.99
scrapy_datetime      2022-04-24 11:00:00-21:30
stlye_id                               1048642
color_id                                     1
color_name                         denim_black
Composition                                NaN
Name: 571, dtype: object

In [102]:
data[data['product_id']== 1048642001]

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
570,1048642001,men_jeans_regular,regular_bootcut_jeans,39.99,2022-04-24 11:00:00-21:30,1048642,1,denim_blue,NaN
571,1048642001,men_jeans_regular,regular_bootcut_jeans,39.99,2022-04-24 11:00:00-21:30,1048642,1,denim_black,NaN


In [103]:
df_aux= data.loc[data['product_id']==1024256004].sort_values('color_name')

df_aux.shape

(7, 9)

In [104]:
df_aux

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition
210,1024256004,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,4,black,NaN
213,1024256004,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,4,dark_blue,NaN
209,1024256004,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,4,dark_denim_blue,NaN
214,1024256004,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,4,dark_gray,NaN
212,1024256004,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,4,denim_blue,NaN
211,1024256004,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,4,light_denim_blue,NaN
215,1024256004,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,4,white,NaN


In [59]:
df_aux.apply(lambda x: len(x.unique()))

product_id           1
products_category    1
products_name        1
products_price       1
scrapy_datetime      1
stlye_id             1
color_id             1
color_name           7
Composition          1
dtype: int64

In [7]:
#Cotton| Spandex | Polyester |  Elastomultiester
#2 etapa
#definir as colunas (organizando as colunas por máteria-prima)

#criando Dataframe vazio com o mesmo tamanho do dataframe original para prencher com as respectivas composições
df_ref = pd.DataFrame(index=np.arange(len(data) ),columns=['cotton','spandex','polyester','elastomultiester'])

#cotton
#pegando a coluna da composição cotton
df_cotton = df1[0]

#colocando o nome na coluna 
df_cotton.name = 'cotton'
#mesclando com a tabela vazia 
df_ref = pd.concat([df_ref,df_cotton],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#polyester 

#adicionando as coluna da composição Polyester
df_polyester =df1[1].where(lambda x: x.str.contains('Polyester',na=True))

#renomeando a coluna
df_polyester.name= 'polyester'

df_ref= pd.concat([df_ref,df_polyester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#Elastomultiester

#coletando apenas a composição elastano
df_elastomultiester =df1[1].where(lambda x: x.str.contains('Elastomultiester',na=True))

#renomenado a coluna eslastano
df_elastomultiester.name= 'elastomultiester'

#mesclando com df_ref vazio e com a mesma quantidade de linhas que há no index
df_ref= pd.concat([df_ref,df_elastomultiester],axis=1)

#dropando as colunas duplicadas
df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#Spandex

#coletando composição spandex da segunda coluna df1

df_spandex_1= df1[1].where(lambda x: x.str.contains('Spandex',na=True))

df_spandex_1.name = 'spandex'

df_ref= pd.concat([df_ref,df_spandex_1],axis=1)

df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

#coletando composição spandex da terceira coluna do df1

df_spandex_2= df1[2].where(lambda x: x.str.contains('Spandex',na=True))
             
df_spandex_2.name= 'spandex'

df_ref= pd.concat([df_ref,df_spandex_2],axis=1)

df_ref= df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]



#final join

data= pd.concat([data,df_ref],axis=1)

#coletando apenas os numeros e transformando em números inteiros

#format composition data

#cotton
data['cotton']= data['cotton'].apply(lambda x: int(re.search('\d+',x ).group(0))/100 if pd.notnull(x) else x)

#spandex
data['spandex'] =data['spandex'].apply(lambda x: int(re.search('\d+', x).group(0)) /100 if pd.notnull(x) else x)

#polyester
data['polyester']= data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

#elastomultiester
data['elastomultiester']=data['elastomultiester'].apply(lambda x: int(re.search('\d+', x).group(0)) /100 if pd.notnull(x) else x)

In [8]:
data.sample(20)

,product_id,products_category,products_name,products_price,scrapy_datetime,stlye_id,color_id,color_name,Composition,cotton,polyester,elastomultiester,spandex
2966,938875010,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-24 11:00:00-21:30,938875,10,pale_denim_blue,NaN,NaN,NaN,NaN,NaN
2538,938875007,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-24 11:00:00-21:30,938875,7,denim_blue,"Cotton 99%, Spandex 1%",0.99,NaN,NaN,NaN
5047,875105024,men_jeans_relaxed,relaxed_jeans,29.99,2022-04-24 11:00:00-21:30,875105,24,dark_denim_blue,NaN,NaN,NaN,NaN,NaN
404,938875002,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-24 11:00:00-21:30,938875,2,dark_gray,"Cotton 99%, Spandex 1%",0.99,NaN,NaN,NaN
4892,1008110003,men_jeans_slim,freefit®_slim_jeans,49.99,2022-04-24 11:00:00-21:30,1008110,3,dark_denim_blue,NaN,NaN,NaN,NaN,NaN
3662,1024256006,men_jeans_slim,slim_jeans,19.99,2022-04-24 11:00:00-21:30,1024256,6,dark_blue,NaN,NaN,NaN,NaN,NaN
7115,875105002,men_jeans_relaxed,relaxed_jeans,29.99,2022-04-24 11:00:00-21:30,875105,2,dark_denim_blue,NaN,NaN,NaN,NaN,NaN
3413,985159008,men_jeans_skinny,skinny_jeans,19.99,2022-04-24 11:00:00-21:30,985159,8,denim_blue,NaN,NaN,NaN,NaN,NaN
262,938875013,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-24 11:00:00-21:30,938875,13,denim_blue,"Cotton 99%, Spandex 1%",0.99,NaN,NaN,NaN
972,690449036,men_jeans_ripped,skinny_jeans,39.99,2022-04-24 11:00:00-21:30,690449,36,black_trashed,"Cotton 98%, Spandex 2%",0.98,NaN,NaN,NaN
